<a href="https://colab.research.google.com/github/Jacin21/Delivery_Charge_Analysis/blob/main/Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING THE PACKAGES

In [ ]:
import pandas as pd

LOADING THE DATASETS

In [ ]:
order_report = pd.read_excel('Company X - Order Report.xlsx') 
order_report.head()

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


In [ ]:
order_report.shape

(400, 3)

In [ ]:
order_report = order_report.rename(columns={'ExternOrderNo': 'Order ID'})

In [ ]:
pincode_zone=pd.read_excel('/content/Company X - Pincode Zones.xlsx')
pincode_zone.head() 

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d


In [ ]:
sku_master=pd.read_excel('/content/Company X - SKU Master.xlsx')
sku_master.head()

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


In [ ]:
sku_master.shape

(66, 2)

In [ ]:
invoice=pd.read_excel('/content/Courier Company - Invoice.xlsx')
invoice.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4


In [ ]:
rates=pd.read_excel('/content/Courier Company - Rates.xlsx')
rates.head()

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


In [ ]:
order_report['weight(g)']=sku_master['Weight (g)']
order_report.head()

,Order ID,SKU,Order Qty,weight(g)
0,2001827036,8904223818706,1.0,210.0
1,2001827036,8904223819093,1.0,165.0
2,2001827036,8904223819109,1.0,113.0
3,2001827036,8904223818430,1.0,65.0
4,2001827036,8904223819277,1.0,120.0


In [ ]:
pincode_zone['Customer Pincode'].duplicated().sum()

16

In [ ]:
pincode_zone=pincode_zone.drop_duplicates(subset=['Customer Pincode'])

In [ ]:
Company_invoice = invoice[['Order ID', 'Customer Pincode','Type of Shipment']]

Merging columns.

In [ ]:
merged_Pincode = Company_invoice.merge(pincode_zone, on = 'Customer Pincode') 
merged_Pincode.head()

,Order ID,Customer Pincode,Type of Shipment,Warehouse Pincode,Zone
0,2001806232,507101,Forward charges,121003,d
1,2001806273,486886,Forward charges,121003,d
2,2001806408,532484,Forward charges,121003,d
3,2001806458,143001,Forward charges,121003,b
4,2001807012,515591,Forward charges,121003,d


In [ ]:
merged_Courier = order_report.merge(merged_Pincode, on = 'Order ID')
merged_Courier.head()

,Order ID,SKU,Order Qty,weight(g),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone
0,2001827036,8904223818706,1.0,210.0,173213,Forward charges,121003,e
1,2001827036,8904223819093,1.0,165.0,173213,Forward charges,121003,e
2,2001827036,8904223819109,1.0,113.0,173213,Forward charges,121003,e
3,2001827036,8904223818430,1.0,65.0,173213,Forward charges,121003,e
4,2001827036,8904223819277,1.0,120.0,173213,Forward charges,121003,e


In [ ]:
merged_Courier.shape

(400, 8)

In [ ]:
merged_Courier.isnull().sum()

Order ID               0
SKU                    0
Order Qty              0
weight(g)            334
Customer Pincode       0
Type of Shipment       0
Warehouse Pincode      0
Zone                   0
dtype: int64

In [ ]:
merged_Courier = merged_Courier.fillna(merged_Courier.mean())
merged_Courier.sample()

<ipython-input-18-83d24f54bc27>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  merged_Courier = merged_Courier.fillna(merged_Courier.mean())


,Order ID,SKU,Order Qty,weight(g),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone
37,2001821185,8904223819253,1.0,240.0,313001,Forward charges,121003,b


In [ ]:
merged_Courier.isnull().sum()

Order ID             0
SKU                  0
Order Qty            0
weight(g)            0
Customer Pincode     0
Type of Shipment     0
Warehouse Pincode    0
Zone                 0
dtype: int64

In [ ]:
merged_Courier['Total_Weight'] = Total_Weight=(merged_Courier['Order Qty']*merged_Courier['weight(g)'])/1000
merged_Courier.sample()

,Order ID,SKU,Order Qty,weight(g),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone,Total_Weight
235,2001808118,8904223818751,1.0,219.742424,140301,Forward charges,121003,b,0.219742


Creating a function for the calculation of weight slab

In [ ]:
import math
def weight_slab(weight):
    return math.ceil(weight*2)/2

Creating a column for weight slab.

In [ ]:
merged_Courier['Weight Slab(KG)']=merged_Courier['Total_Weight'].apply(weight_slab)
merged_Courier.tail()

,Order ID,SKU,Order Qty,weight(g),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone,Total_Weight,Weight Slab(KG)
395,2001806229,8904223818942,1.0,219.742424,421204,Forward charges,121003,d,0.219742,0.5
396,2001806229,8904223818850,1.0,219.742424,421204,Forward charges,121003,d,0.219742,0.5
397,2001806226,8904223818850,2.0,219.742424,723146,Forward charges,121003,d,0.439485,0.5
398,2001806210,8904223816214,1.0,219.742424,140604,Forward charges,121003,b,0.219742,0.5
399,2001806210,8904223818874,1.0,219.742424,140604,Forward charges,121003,b,0.219742,0.5


In [ ]:
invoice['Weight Slab By Courier Company (KG)']=(invoice['Charged Weight']).apply(weight_slab)
invoice.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Weight Slab By Courier Company (KG)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.5
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,1.0
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2.5
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,1.0
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.5


In [ ]:
# Define a function to calculate the expected charges
def calculate_expected_charges(row):
    fwd_catagory = 'fwd_'+row['Zone']
    fwd_fixed = rates[fwd_catagory+'_fixed'][0]
    fwd_additional = rates[fwd_catagory+'_additional'][0]
    rto_catagory = 'rto_'+row['Zone']
    rto_fixd = rates[rto_catagory+'_fixed'][0]
    rto_additional = rates[rto_catagory+'_additional'][0]
    if row['Type of Shipment'] == 'Forward charges':
        if row['Weight Slab(KG)'] <= 0.5:
            return fwd_fixed
        else:
            additional_weight = (row['Weight Slab(KG)']-0.5)/0.5
            return fwd_fixed+additional_weight*fwd_additional
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        if row['Weight Slab(KG)'] <= 0.5:
            return fwd_fixed+rto_fixd
        else:
            additional_weight=(row['Weight Slab(KG)']-0.5)/0.5
            return fwd_fixed+additional_weight*(fwd_additional+rto_additional)

# Use the apply() function to calculate the expected charges for each row
merged_Courier['Expected Charge As Per X (Rs.)'] = merged_Courier.apply(calculate_expected_charges, axis=1)
merged_Courier.head()

,Order ID,SKU,Order Qty,weight(g),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone,Total_Weight,Weight Slab(KG),Expected Charge As Per X (Rs.)
0,2001827036,8904223818706,1.0,210.0,173213,Forward charges,121003,e,0.210,0.5,56.6
1,2001827036,8904223819093,1.0,165.0,173213,Forward charges,121003,e,0.165,0.5,56.6
2,2001827036,8904223819109,1.0,113.0,173213,Forward charges,121003,e,0.113,0.5,56.6
3,2001827036,8904223818430,1.0,65.0,173213,Forward charges,121003,e,0.065,0.5,56.6
4,2001827036,8904223819277,1.0,120.0,173213,Forward charges,121003,e,0.120,0.5,56.6


Creating output1,output2

In [ ]:
invoice.columns

Index(['AWB Code', 'Order ID', 'Charged Weight', 'Warehouse Pincode',
       'Customer Pincode', 'Zone', 'Type of Shipment', 'Billing Amount (Rs.)',
       'Weight Slab By Courier Company (KG)'],
      dtype='object')

In [ ]:
output1=invoice[['AWB Code', 'Order ID','Charged Weight','Zone','Billing Amount (Rs.)', 'Weight Slab By Courier Company (KG)']]

In [ ]:
merged_Courier.columns

Index(['Order ID', 'SKU', 'Order Qty', 'weight(g)', 'Customer Pincode',
       'Type of Shipment', 'Warehouse Pincode', 'Zone', 'Total_Weight',
       'Weight Slab(KG)', 'Expected Charge As Per X (Rs.)'],
      dtype='object')

In [ ]:
output2=merged_Courier[['Order ID',
       'Type of Shipment', 'Zone',
       'Total_Weight', 'Weight Slab(KG)',
       'Expected Charge As Per X (Rs.)']]

Merging output1 and output2

In [ ]:
merged_output=output2.merge(output1,on='Order ID',how='left')
merged_output.head()

,Order ID,Type of Shipment,Zone_x,Total_Weight,Weight Slab(KG),Expected Charge As Per X (Rs.),AWB Code,Charged Weight,Zone_y,Billing Amount (Rs.),Weight Slab By Courier Company (KG)
0,2001827036,Forward charges,e,0.210,0.5,56.6,1091122418320,1.6,b,117.9,2.0
1,2001827036,Forward charges,e,0.165,0.5,56.6,1091122418320,1.6,b,117.9,2.0
2,2001827036,Forward charges,e,0.113,0.5,56.6,1091122418320,1.6,b,117.9,2.0
3,2001827036,Forward charges,e,0.065,0.5,56.6,1091122418320,1.6,b,117.9,2.0
4,2001827036,Forward charges,e,0.120,0.5,56.6,1091122418320,1.6,b,117.9,2.0


In [ ]:
result1=merged_output.copy()

Rename the **columns**

In [ ]:
result1=result1.rename(columns={'Zone_x':'Delivery Zone as per X',
                                'Total_Weight':'Total Weight AS Per X (KG)',
                                'Weight Slab(KG)':'Weight Slab As Per x (KG)',
                                'AWB Code':'AWB Number',
                                'Charged Weight':'Total weight as per Courier Company (KG)',
                                'Zone_y':'Delivery Zone charged by Courier Company',
                                'Billing Amount (Rs.)':'Charges Billed by Courier Company (Rs.)',
                                'Weight Slab By Courier Company (KG)':'Weight Slab Charged By Courier Company (KG)'})
result1.head()

,Order ID,Type of Shipment,Delivery Zone as per X,Total Weight AS Per X (KG),Weight Slab As Per x (KG),Expected Charge As Per X (Rs.),AWB Number,Total weight as per Courier Company (KG),Delivery Zone charged by Courier Company,Charges Billed by Courier Company (Rs.),Weight Slab Charged By Courier Company (KG)
0,2001827036,Forward charges,e,0.210,0.5,56.6,1091122418320,1.6,b,117.9,2.0
1,2001827036,Forward charges,e,0.165,0.5,56.6,1091122418320,1.6,b,117.9,2.0
2,2001827036,Forward charges,e,0.113,0.5,56.6,1091122418320,1.6,b,117.9,2.0
3,2001827036,Forward charges,e,0.065,0.5,56.6,1091122418320,1.6,b,117.9,2.0
4,2001827036,Forward charges,e,0.120,0.5,56.6,1091122418320,1.6,b,117.9,2.0


In [ ]:
difference_between_expected_and_billed = result1['Expected Charge As Per X (Rs.)'] - result1['Charges Billed by Courier Company (Rs.)']

result1['Difference Between Expected Charges and Billed Charges (Rs.)'] = difference_between_expected_and_billed


In [ ]:
result1.head()

,Order ID,Type of Shipment,Delivery Zone as per X,Total Weight AS Per X (KG),Weight Slab As Per x (KG),Expected Charge As Per X (Rs.),AWB Number,Total weight as per Courier Company (KG),Delivery Zone charged by Courier Company,Charges Billed by Courier Company (Rs.),Weight Slab Charged By Courier Company (KG),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001827036,Forward charges,e,0.210,0.5,56.6,1091122418320,1.6,b,117.9,2.0,-61.3
1,2001827036,Forward charges,e,0.165,0.5,56.6,1091122418320,1.6,b,117.9,2.0,-61.3
2,2001827036,Forward charges,e,0.113,0.5,56.6,1091122418320,1.6,b,117.9,2.0,-61.3
3,2001827036,Forward charges,e,0.065,0.5,56.6,1091122418320,1.6,b,117.9,2.0,-61.3
4,2001827036,Forward charges,e,0.120,0.5,56.6,1091122418320,1.6,b,117.9,2.0,-61.3


Computing Output Data 2

In [ ]:
correct = result1[result1['Difference Between Expected Charges and Billed Charges (Rs.)'] == 0.0]
correctly_charged = ['Total orders where X has been correctly charged', len(correct), correct['Charges Billed by Courier Company (Rs.)'].sum()]

In [ ]:
print(correctly_charged)

['Total orders where X has been correctly charged', 14, 703.8999999999999]


In [ ]:
over=result1[result1['Difference Between Expected Charges and Billed Charges (Rs.)'] < 0.0]
over_charged = ['Total Orders where X has been overcharged', len(over), abs(over['Difference Between Expected Charges and Billed Charges (Rs.)'].sum())]

In [ ]:
over_charged

['Total Orders where X has been overcharged', 375, 30865.700000000004]

In [ ]:
under= result1[result1['Difference Between Expected Charges and Billed Charges (Rs.)'] > 0.0]
under_charged = ['Total Orders where X has been undercharged', len(under), abs(under['Difference Between Expected Charges and Billed Charges (Rs.)'].sum())]

In [ ]:
under_charged

['Total Orders where X has been undercharged', 11, 209.40000000000003]

In [ ]:
result=pd.DataFrame([correctly_charged,over_charged,under_charged],columns=['Category','Count','Amount (Rs.)'])

In [ ]:
result

,Category,Count,Amount (Rs.)
0,Total orders where X has been correctly charged,14,703.9
1,Total Orders where X has been overcharged,375,30865.7
2,Total Orders where X has been undercharged,11,209.4


Loading the Dataframes into a Excel file.

In [ ]:
Excel = pd.ExcelWriter('Expected_Result.xlsx', engine='openpyxl')
result1.to_excel(Excel, sheet_name='Calculations', index=False)
Calculations=Excel.save()

In [ ]:
result.to_excel(Excel,sheet_name='Summary')
Excel.save()

In [ ]:
sheets = pd.read_excel('Expected_Result.xlsx', sheet_name=['Calculations', 'Summary'])
sheets['Calculations']

,Order ID,Type of Shipment,Delivery Zone as per X,Total Weight AS Per X (KG),Weight Slab As Per x (KG),Expected Charge As Per X (Rs.),AWB Number,Total weight as per Courier Company (KG),Delivery Zone charged by Courier Company,Charges Billed by Courier Company (Rs.),Weight Slab Charged By Courier Company (KG),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001827036,Forward charges,e,0.210000,0.5,56.6,1091122418320,1.60,b,117.9,2.0,-61.3
1,2001827036,Forward charges,e,0.165000,0.5,56.6,1091122418320,1.60,b,117.9,2.0,-61.3
2,2001827036,Forward charges,e,0.113000,0.5,56.6,1091122418320,1.60,b,117.9,2.0,-61.3
3,2001827036,Forward charges,e,0.065000,0.5,56.6,1091122418320,1.60,b,117.9,2.0,-61.3
4,2001827036,Forward charges,e,0.120000,0.5,56.6,1091122418320,1.60,b,117.9,2.0,-61.3
...,...,...,...,...,...,...,...,...,...,...,...,...
395,2001806229,Forward charges,d,0.219742,0.5,45.4,1091117222080,0.71,d,90.2,1.0,-44.8
396,2001806229,Forward charges,d,0.219742,0.5,45.4,1091117222080,0.71,d,90.2,1.0,-44.8
397,2001806226,Forward charges,d,0.439485,0.5,45.4,1091117222065,0.68,d,90.2,1.0,-44.8
398,2001806210,Forward charges,b,0.219742,0.5,33.0,1091117221940,2.92,b,174.5,3.0,-141.5


In [ ]:
sheets['Summary']

,Unnamed: 0,Category,Count,Amount (Rs.)
0,0,Total orders where X has been correctly charged,14,703.9
1,1,Total Orders where X has been overcharged,375,30865.7
2,2,Total Orders where X has been undercharged,11,209.4
